В данном файле происходит дообучение модели bert-base-multilingual-cased с Hugging Face. В качестве токенизатора используется bert-base-multilingual-cased.

In [ ]:
import pandas as pd
import torch
import pandas as pd
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
from tqdm import tqdm


Загружаем готовый датасет из предыдущего файла

In [ ]:
text_data = pd.read_csv('/content/text_data.csv')
text_data.tail()

,text,no_space
211242,Создано средство от преждевременной смерти,Созданосредствоотпреждевременнойсмерти
211243,У активных женщин,Уактивныхженщин
211244,Черчесов остался в сборной России,ЧерчесовосталсявсборнойРоссии
211245,Белоруссия поставила под сомнение торговый сою...,БелоруссияпоставилаподсомнениеторговыйсоюзсРос...
211246,Российское посольство подкололо Терезу Мэй мем...,РоссийскоепосольствоподкололоТерезуМэймемомпро...


In [ ]:
def make_labels(df):
    no_space = df["no_space"]
    with_space = df["text"]

    labels = []
    j = 0
    for i, ch in enumerate(no_space):
        labels.append(0)
        j += 1
        if j < len(with_space) and with_space[j] == " ":
            labels[-1] = 1
            j += 1
    df["labels"] = labels
    return df

Создаем датасет HuggingFace и подгружаем предобученныый токенизатор bert-base-multilingual-cased

In [ ]:
dataset = Dataset.from_pandas(text_data)

In [ ]:
dataset = dataset.map(make_labels)

Map:   0%|          | 0/211247 [00:00<?, ? examples/s]

Вид нового датасета для обучения

In [ ]:
df_temp = pd.DataFrame(dataset[:10])
df_temp.head()

,text,no_space,labels
0,Мы по голосу решали,Мыпоголосурешали,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
1,В автономном учреждении,Вавтономномучреждении,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
2,В России осудили серийных охотников за пиццей,ВРоссииосудилисерийныхохотниковзапиццей,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,"Убежало,","Убежало,","[0, 0, 0, 0, 0, 0, 0, 0]"
4,Убитого сотрудника МВД нашли в багажнике угнан...,УбитогосотрудникаМВДнашливбагажникеугнанноймаш...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
    """
    Получает на вход батч (словарь списков) с полями:
    - "no_space": строки без пробелов
    - "labels": списки меток (0/1) по символам для каждой строки

    Возвращает словарь encodings
    """

    tokens = [list(x) for x in batch["no_space"]]
    encodings = tokenizer(tokens, is_split_into_words=True,
                          padding="max_length", truncation=True, max_length=128)

    encodings["labels"] = [
        l + [-100] * (128 - len(l)) for l in batch["labels"]
    ]
    return encodings

In [ ]:
dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/211247 [00:00<?, ? examples/s]

Разбиваем датасет на train и test, с размеров тестовой выборки ~ 22000 строк.

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

Инициализурем модель bert-base-multilingual-cased, также кроме нее пробовались следующие модели, но результат показали хуже:

*   DeepPavlov/rubert-base-cased
*   xlm-roberta-base
*   sberbank-ai/ruBert-base





In [ ]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=2
)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Обучающие аргументы:

*   per_device_  train/eval  _batch_size=16/ - число примеров в 1 батче
*   Каждый 5000 шагов модель сохраняет последнюю версию и происходит оценка качества. При этом хранится только последняя версия
*   Количество эпох - 7





In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=5000,
    report_to="none",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    save_strategy="steps",
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=50
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
)

/tmp/ipython-input-3444495420.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Для обучении использовалась GPU A100 в GOOGLE COLAB.
*   40 GB видеопамяти
*   80 GB оперативной памяти




In [ ]:
trainer.train()

In [ ]:
import pandas as pd

def load_txt_to_df(file_path: str, sep: str = ",", columns=("id", "text")) -> pd.DataFrame:
    """
    Загружает txt-файл в DataFrame.
    Ожидается формат: id,text.

    Принимает:
        file_path (str): путь к txt-файлу
        sep (str): разделитель (по умолчанию ",")
        columns: имена колонок (по умолчанию ("id", "text"))

    Возвращает:
        pd.DataFrame
    """
    with open(file_path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f]

    data = [line.split(sep, 1) for line in lines]
    df = pd.DataFrame(data, columns=columns)
    df = df.iloc[1:].reset_index(drop=True)
    return df


Для оценки качества загружаем два файла txt: тестовые данные и тестовые данные с проставленными пробелами.

In [ ]:
test_NS_path = '/content/dataset_1937770_3.txt'

test_no_spaces_df = load_txt_to_df(test_NS_path)

In [ ]:
test_df_path = '/content/almost_trooth.txt'
test_df = load_txt_to_df(test_df_path)

In [ ]:
test_df.tail()

,id,text
1000,1000,Я не усну.
1001,1001,Весна - я уж не грею пио.
1002,1002,Весна - скоро вырастет трава.
1003,1003,"Весна - вы посмотрите, как красиво."
1004,1004,Весна - где моя голова?


In [ ]:
test_no_spaces_df.tail()

,id,text
1000,1000,Янеусну.
1001,1001,Весна-яуженегреюпио.
1002,1002,Весна-скоровырастеттрава.
1003,1003,"Весна-выпосмотрите,каккрасиво."
1004,1004,Весна-гдемояголова?


In [ ]:
def restore_spaces(text, model, tokenizer, device="cuda"):
    """
    Восстанавливает пробелы в тексте по предсказаниям модели.

    Принимает:
        text (str): строка без пробелов
        model: обученная модель
        tokenizer: токенизатор
        device (str): устройство для вычислений ("cuda" или "cpu")

    Возвращает:
        str: восстановленный текст с пробелами
    """

    model.to(device)
    model.eval()

    tokens = list(text)
    enc = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", truncation=True, max_length=128)
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        outputs = model(**enc)
        preds = outputs.logits.argmax(-1).squeeze().tolist()

    result = ""
    for ch, p in zip(tokens, preds[:len(tokens)]):
        result += ch
        if p == 1:
            result += " "
    return result


Сделаем предсказания для тестовых данных самого задания

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

test_no_spaces_df['text_spaces'] = [
    restore_spaces(text, model, tokenizer, device=device)
    for text in tqdm(test_no_spaces_df['text'], desc="Restoring spaces")
]


Restoring spaces: 100%|██████████| 1005/1005 [00:11<00:00, 86.91it/s]


In [ ]:
test_no_spaces_df.head()

,id,text,text_spaces
0,0,куплюайфон14про,куплю айфон 14 про
1,1,ищудомвПодмосковье,ищу дом в Подмосковье
2,2,сдаюквартирусмебельюитехникой,сдаю квартиру с мебелью и техникой
3,3,новыйдивандоставканедорого,новый диван доставка недорого
4,4,отдамдаромкошку,отдам даром кошку


In [ ]:
def get_space_indices(text):
    """
    Возвращает индексы всех пробелов в строке

    Принимает:
        text (str): строка с пробелами

    Возвращает:
        list[int]: список индексов (позиции символов в строке),
    """
    return [i for i, ch in enumerate(text) if ch == " "]

In [ ]:
# Поулчаем индексы пробелов для тестового текста, в котором пробелы были проставлены с помощью предсказаний модели
test_no_spaces_df['predicted_positions'] = test_no_spaces_df['text_spaces'].apply(get_space_indices)

In [ ]:
# Получаем индексы пробелов для тестового текста, с проставленными пробелами вручную
test_df['predicted_positions'] = test_df['text'].apply(get_space_indices)

In [ ]:
test_df.head()

,id,text,predicted_positions
0,0,куплю айфон 14 про,"[5, 11, 14]"
1,1,ищу дом в Подмосковье,"[3, 7, 9]"
2,2,сдаю квартиру с мебелью и техникой,"[4, 13, 15, 23, 25]"
3,3,новый диван доставка недорого,"[5, 11, 20]"
4,4,отдам даром кошку,"[5, 11]"


In [ ]:
test_no_spaces_df.head()

,id,text,text_spaces,predicted_positions
0,0,куплюайфон14про,куплю айфон 14 про,"[5, 11, 14]"
1,1,ищудомвПодмосковье,ищу дом в Подмосковье,"[3, 7, 9]"
2,2,сдаюквартирусмебельюитехникой,сдаю квартиру с мебелью и техникой,"[4, 13, 15, 23, 25]"
3,3,новыйдивандоставканедорого,новый диван доставка недорого,"[5, 11, 20]"
4,4,отдамдаромкошку,отдам даром кошку,"[5, 11]"


In [ ]:
import pandas as pd

def f1_spaces(df_true: pd.DataFrame, df_pred: pd.DataFrame,
              true_col: str = "predicted_positions",
              pred_col: str = "predicted_positions") -> float:
    """
    Считает средний F1 для задачи восстановления пробелов.

    Принимает:
        df_true (pd.DataFrame): датафрейм с истинными позициями
        df_pred (pd.DataFrame): датафрейм с предсказанными позициями
        true_col (str): имя колонки с правильными позициями
        pred_col (str): имя колонки с предсказанными позициями

    Возвращает:
        float: средний F1 (0-1)
    """
    f1_scores = []

    for true_pos, pred_pos in zip(df_true[true_col], df_pred[pred_col]):
        if isinstance(true_pos, str):
            true_pos = eval(true_pos)
        if isinstance(pred_pos, str):
            pred_pos = eval(pred_pos)

        true_set, pred_set = set(true_pos), set(pred_pos)

        tp = len(true_set & pred_set)
        fp = len(pred_set - true_set)
        fn = len(true_set - pred_set)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0

        f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0
        f1_scores.append(f1)

    return sum(f1_scores) / len(f1_scores)


In [ ]:
print("F1:", f1_spaces(test_df, test_no_spaces_df))

Сохраняем и архивируем модель

In [ ]:
trainer.save_model("./model")
tokenizer.save_pretrained("./model")

('./my_trained_model/tokenizer_config.json',
 './my_trained_model/special_tokens_map.json',
 './my_trained_model/vocab.txt',
 './my_trained_model/added_tokens.json',
 './my_trained_model/tokenizer.json')

In [ ]:
!zip -r model.zip ./model

  adding: my_trained_model/ (stored 0%)
  adding: my_trained_model/model.safetensors (deflated 7%)
  adding: my_trained_model/training_args.bin (deflated 54%)
  adding: my_trained_model/vocab.txt (deflated 45%)
  adding: my_trained_model/config.json (deflated 54%)
  adding: my_trained_model/tokenizer_config.json (deflated 75%)
  adding: my_trained_model/tokenizer.json (deflated 67%)
  adding: my_trained_model/special_tokens_map.json (deflated 42%)
